In [1]:
!dir

 Volume in drive C is OS
 Volume Serial Number is ECFF-C11A

 Directory of C:\Github\Linear-Equation-Solver

07-07-2019  21:43    <DIR>          .
07-07-2019  21:43    <DIR>          ..
07-07-2019  20:20    <DIR>          .ipynb_checkpoints
07-07-2019  21:43            10,083 Dataset creation.ipynb
07-07-2019  20:20           199,556 division and equals merging.ipynb
07-07-2019  20:20            13,910 division_testing.jpg
07-07-2019  20:20           209,151 Final extracting individual symbols.ipynb
07-07-2019  20:20            25,167 hard_test.jpg
07-07-2019  20:20            36,177 random_tests.jpg
07-07-2019  20:20                29 README.md
07-07-2019  20:20            20,800 simple_equation.jpg
07-07-2019  20:20           259,741 testing letter detection.ipynb
               9 File(s)        774,614 bytes
               3 Dir(s)  297,145,876,480 bytes free


In [8]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import os
import cv2 as cv
import pandas as pd
from sklearn.model_selection import  train_test_split
from keras import models
from keras import layers
from keras.utils import to_categorical
import tensorflow as tf
import keras

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7435153314445385025
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3078304563
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2288630952599019602
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [10]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [11]:
def disp_img(img):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    plt.show()

In [12]:
mpl.rcParams['figure.dpi']= 180

In [13]:
data_dir = './dataset'
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', 'x', 'div', 'fslash', '=', '(', ')']

In [14]:
num_classes = len(labels)
print(f'Number of classes = {num_classes}')

Number of classes = 18


In [15]:
dataset = []

In [16]:
for label in labels:
    path = os.path.join(data_dir, label)
    label_val = labels.index(label)
    for img in os.listdir(path):
        pad = 1
        gray = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
        blurred = cv.GaussianBlur(gray, (3, 3), 5)
        thresh = cv.adaptiveThreshold(blurred, 255, 0, 1, 115, 1)
        padded = cv.copyMakeBorder(thresh, top=pad, bottom=pad, left=pad, right=pad, borderType= cv.BORDER_CONSTANT, value=[0,0,0] )
        dataset.append([padded, label_val])

FileNotFoundError: [WinError 3] The system cannot find the path specified: './dataset\\0'

In [56]:
len(dataset)

199437

In [ ]:
df = pd.DataFrame(dataset, columns=list('Xy'))
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head(100)

In [ ]:
X = dataset.X
y = dataset.y

In [ ]:
g = sns.countplot(y_train)

In [ ]:
X = X.values.reshape(-1, 45, 45, 1)

In [ ]:
y = to_categorical(y, num_classes = num_classes)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (46, 46, 1)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 20, batch_size = 128, validation_data = (X_val, y_val), verbose = 2)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

In [ ]:
epochs = range(1, 21)

plt.plot(epochs, loss, 'ko', label = 'Training Loss')
plt.plot(epochs, val_loss, 'k', label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
plt.plot(epochs, acc, 'yo', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'y', label = 'Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
results = model.predict(X_test)

In [ ]:
results = np.argmax(results, axis = 1)
results = pd.Series(results, name = 'Label')

In [ ]:
results

In [ ]:
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")